In [3]:
!mkdir -p scripts

In [4]:
g = "raw.githubusercontent.com"
p = "PacktPublishing"
a = "Amazon-SageMaker-Cookbook"
mc = "master/Chapter09"

path = f"https://{g}/{p}/{a}/{mc}/scripts"

In [5]:
!wget -P scripts {path}/setup.py
!wget -P scripts {path}/train.py
!wget -P scripts {path}/inference.py
!wget -P scripts {path}/requirements.txt

--2021-06-08 02:29:04--  https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Chapter09/scripts/setup.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 181 [text/plain]
Saving to: ‘scripts/setup.py.1’

setup.py.1          100%[===================>]     181  --.-KB/s    in 0s      

2021-06-08 02:29:04 (6.00 MB/s) - ‘scripts/setup.py.1’ saved [181/181]

--2021-06-08 02:29:04--  https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Chapter09/scripts/train.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awa

In [6]:
!mkdir -p tmp

In [7]:
g = "raw.githubusercontent.com"
p = "PacktPublishing"
a = "Amazon-SageMaker-Cookbook"
mc = "master/Chapter09"

path = f"https://{g}/{p}/{a}/{mc}/files"

In [8]:
!wget -P tmp {path}/synthetic.train.txt

--2021-06-08 02:29:06--  https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Chapter09/files/synthetic.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154276 (151K) [text/plain]
Saving to: ‘tmp/synthetic.train.txt.1’

synthetic.train.txt 100%[===================>] 150.66K  --.-KB/s    in 0.004s  

2021-06-08 02:29:06 (34.1 MB/s) - ‘tmp/synthetic.train.txt.1’ saved [154276/154276]



In [9]:
!wget -P tmp {path}/synthetic.validation.txt

--2021-06-08 02:29:06--  https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Chapter09/files/synthetic.validation.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50653 (49K) [text/plain]
Saving to: ‘tmp/synthetic.validation.txt.1’

synthetic.validatio 100%[===================>]  49.47K  --.-KB/s    in 0.001s  

2021-06-08 02:29:06 (94.0 MB/s) - ‘tmp/synthetic.validation.txt.1’ saved [50653/50653]



In [10]:
s3_bucket = "sagemaker-cookbook-bucket"
prefix = "chapter09"

In [11]:
s3_train_data = 's3://{}/{}/input/{}'.format(
    s3_bucket, 
    prefix, 
    "synthetic.train.txt"
)
s3_validation_data = 's3://{}/{}/input/{}'.format(
    s3_bucket, 
    prefix, 
    "synthetic.validation.txt"
)

In [12]:
!aws s3 cp tmp/synthetic.train.txt {s3_train_data}

upload: tmp/synthetic.train.txt to s3://sagemaker-cookbook-bucket/chapter09/input/synthetic.train.txt


In [13]:
!aws s3 cp tmp/synthetic.validation.txt {s3_validation_data}

upload: tmp/synthetic.validation.txt to s3://sagemaker-cookbook-bucket/chapter09/input/synthetic.validation.txt


In [14]:
import sagemaker
from sagemaker import Session

role = sagemaker.get_execution_role()
session = sagemaker.Session()

In [15]:
from sagemaker.huggingface import HuggingFace

hyperparameters = {
    'epochs': 1,
    'train_batch_size': 32,
    'model_name':'distilbert-base-uncased'
}

In [16]:
estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./scripts',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=role,
    transformers_version='4.4',
    pytorch_version='1.6',
    py_version='py36',
    hyperparameters=hyperparameters
)

In [17]:
from sagemaker.inputs import TrainingInput

train_data = TrainingInput(s3_train_data)
validation_data = TrainingInput(s3_validation_data)

data_channels = {
    'train': train_data, 
    'valid': validation_data
}

In [18]:
%%time

estimator.fit(data_channels)

2021-06-08 02:29:10 Starting - Starting the training job...
2021-06-08 02:29:13 Starting - Launching requested ML instancesProfilerReport-1623119350: InProgress
......
2021-06-08 02:30:26 Starting - Preparing the instances for training.........
2021-06-08 02:32:07 Downloading - Downloading input data...
2021-06-08 02:32:27 Training - Downloading the training image.................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-06-08 02:35:19,571 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-06-08 02:35:19,594 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-06-08 02:35:21,012 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-06-08 02:35:21,329 sagemaker-training-toolkit INFO     Installing module with the following command:
/opt/conda/bin/python3.6 -m pip install . -r requirements.txt
Processi

In [19]:
from sagemaker.pytorch.model import PyTorchModel

model_data = estimator.model_data

model = PyTorchModel(
    model_data=model_data, 
    role=role, 
    source_dir="scripts",
    entry_point='inference.py', 
    framework_version='1.6.0',
    py_version="py3"
)

In [20]:
%%time

predictor = model.deploy(
    instance_type='ml.m5.xlarge', 
    initial_instance_count=1
)

---------------!CPU times: user 18.2 s, sys: 3.34 s, total: 21.5 s
Wall time: 7min 52s


In [21]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

In [22]:
test_data = {
    "text": "This tastes bad. I hate this place."
}

predictor.predict(test_data)

'NEGATIVE'

In [23]:
test_data = {
    "text": "Very delicious. I would recommend this to my friends"
}

predictor.predict(test_data)

'POSITIVE'

In [24]:
predictor.delete_endpoint()